# Description of the network and installed capacities before the optimization

In [34]:
import logging
import os

import pypsa
import yaml
import pandas as pd
import geopandas as gpd
import geoviews as gv
import hvplot.pandas
import numpy as np
import scipy as sp
import networkx as nx

# plotting stuff
import matplotlib.pyplot as plt
plt.style.use("bmh")
import seaborn as sns  ###
import cartopy.crs as ccrs
sns.set(style='darkgrid')

from scipy.sparse import csgraph
from itertools import product

from shapely.geometry import Point, LineString
import shapely, shapely.prepared, shapely.wkt

logger = logging.getLogger(__name__)

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 70)

import sys
sys.path.append('../')  # to import helpers
from scripts._helpers import _sets_path_to_root
_sets_path_to_root("pypsa-africa")

## load network before optimization

In [35]:
input_solve_network = os.getcwd() + '/networks/elec_s_10_ec_lv0.3_Co2L-3H.nc'

n_input = pypsa.Network(input_solve_network)

## Overview of the inputs and installed capacities

### Number of components by network type

In [36]:
for c in n_input.iterate_components(list(n_input.components.keys())[2:]):
    print("Component '{}' has {} entries".format(c.name,len(c.df)))

### Overview of the characteristics of the network
Below we describe the main characteristics of the network are drawn:
- bus0: the first ending of the line
- bus1: the second ending of the line
- s_nom: the nominal capacity of the line
- capital_cost: the investment cost of the line
- s_nom_extendable: (True/False) binary value stating whether the line capacity can be upgraded

In [37]:
n_input.lines[['bus0', 'bus1', 's_nom', 'capital_cost', 's_nom_extendable']]

### Overview of the installed generation by carrier
Below we describe the main installed capacities (p_nom_opt), average specific capital cost (capital_cost) and average marginal cost (marginal_cost) by carrier and bus
- p_nom_opt: nominal installed capacity
- capital_cost: investment cost of the technology
- marginal_cost: marginal cost of the technology
- p_nom_extendable: (True/False) binary variable stating whether the capacity of the asset can be changed

In [38]:
gen_cap = n_input.generators.groupby(['bus', 'carrier'])[
    ['p_nom_opt', 'capital_cost', 'marginal_cost', 'p_nom_extendable']
    ].agg({'p_nom_opt': 'sum', 'capital_cost': 'mean', 'marginal_cost': 'mean', 'p_nom_extendable': 'sum'})
gen_cap

### Average specific energy production by node and carrier
In this picture, we detail the specific energy production by bus and carrier; the output is a panda Series
In the first column (first index) the bus name is shown; in the second column (second index), the carrier is specified and finally the energy quantity is detailed in the last column

In [39]:
n_input.generators_t.p_max_pu.sum().rename('en_pu')

### Plot of the demand by bus
In this cells, the total demand by bus is created and denoted as `load_serie`, then it is plotted to show the location of the demand.
In the plot, the size of the dot denotes the total demand and the size of the lines in the network denotes the total network capacity between the nodes.

In [40]:
# Creation of the load series by bus
load_serie = n_input.loads_t.p_set.sum().rename("load").rename_axis("bus")

In [41]:
# plot the load serie

fig,ax = plt.subplots(
        figsize=(10,10),
        subplot_kw={"projection":ccrs.PlateCarree()} #PlateCarree(default), Mercator, Orthographic
)

n_input.plot(margin=0.1, 
        ax=None, 
        geomap=True, #'10m' leads to a beautiful high resolution map 
        projection= None, 
        bus_colors='cadetblue', 
        bus_alpha=1, 
        bus_sizes=load_serie*.0000003, #n.loads_t.p_set.sum().rename("load").rename_axis("bus")*.0000003, 
        bus_cmap=None, 
        line_colors='rosybrown', 
        link_colors='darkseagreen', #loading
        transformer_colors='orange', 
        line_widths=n_input.lines.s_nom/3e2, 
        link_widths=1.5, 
        transformer_widths=1.5, 
        line_cmap=None, 
        link_cmap=None, #plt.cm.virdis (colordmap for line loading)
        transformer_cmap=None, 
        flow=None, 
        branch_components=None, 
        layouter=None, 
        title='Generator Map', 
        boundaries=None, 
        geometry=False, 
        jitter=None, 
        color_geomap=True)#None or True

ax.axis('off');